In [14]:
import os
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import shutil

In [15]:
TEST_PATH = '/home/jupyter/test_images/'
TEST_L_PATH = '/home/jupyter/test_labels/'

FRAME_PATH = os.path.join(os.getcwd(), 'images') 

MASK_PATH = os.path.join(os.getcwd(), 'labels')

test_img_list_path = os.path.join(os.getcwd(), 'metadata/test_list.txt')

In [16]:
def save_image(TEST_PATH, image_id):
    
    try:
        if os.path.isfile(os.path.join(TEST_PATH, "{}.jpg".format(image_id)))==True:
            return
        else:
            img_fp = os.path.join(FRAME_PATH, "{}.jpg".format(image_id))
            image = cv2.imread(img_fp)
            
            img_ts = os.path.join(TEST_PATH, "{}.jpg".format(image_id))

            cv2.imwrite(img_ts, image)
    except:
        #images missing that we won't add to the model

        return image_id

In [17]:
def save_label(TEST_L_PATH, image_id):
    
    shutil.copy2(f'{MASK_PATH}/{image_id}.txt', f'{TEST_L_PATH}/{image_id}.txt')


In [11]:
with open(test_img_list_path, "r") as test_img_list_f:
    # tqdm run up to 1000(The # of test set)
    for line in tqdm(test_img_list_f):
        #print(line.strip().encode())

        image_id = line.strip().split('/')[-1].strip().split('.')[0]

        save_image(TEST_PATH, image_id)
        save_label(TEST_L_PATH, image_id)
        

1302it [00:00, 73682.57it/s]
